In [2]:
import xgboost as xgb

```python
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']

# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = gbm.predict(test_X)
```

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt

trajectories = {} ##加载的每个路线的平均运行时间数据
volume = {}

In [4]:
def LoadData():
    df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')
    start_time  = datetime.strptime("2016-07-19 00:00:00",'%Y-%m-%d %H:%M:%S')

    for idx in range(df.shape[0]):
        line = df.iloc[idx]
        route = str(line['intersection_id']+str(line['tollgate_id']))
        stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
        day = (stime - start_time).days
        tw_n = (stime.hour * 60 + stime.minute)/20

        if route not in trajectories.keys():
            trajectories[route] = np.zeros(72*91).reshape(91,72)
        trajectories[route][day][tw_n] = line['avg_travel_time']

In [7]:
LoadData()

In [19]:
train_X = trajectories['A2'][0:-7,18:24]
train_y = trajectories['A2'][0:-7,24]

In [39]:
gbm = xgb.XGBRegressor(max_depth=3,n_estimators=300,learning_rate=0.05).fit(train_X,train_y)

In [49]:
test_X = trajectories['A2'][-6,18:24].reshape(1,6)

In [50]:
gbm.predict(test_X)

array([ 86.44766998], dtype=float32)

In [52]:
trajectories['A2'][-6,24]

86.159999999999997

In [58]:
test_X = np.zeros(12)
test_X[0:6] = trajectories['A2'][-6,18:24]

for i in range(6,12):
    test_X[i] = gbm.predict(test_X[i-6:i].reshape(1,6))

In [63]:
test_X[6:12]

array([  86.44766998,   85.31115723,   93.8404007 ,  100.13945007,
         69.02466583,   67.7482605 ])

In [62]:
trajectories['A2'][-6,24:30]

array([ 86.16,  83.28,  90.04,  67.48,  69.26,  65.32])